In [3]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


In [5]:
TRAINING_SIZE = 50000
DIGITS = 3

REVERSE = True

MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+ '

RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1


# Encoder & Decoder

In [6]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)


In [7]:
ctable = CharacterTable(chars)


# Data Generation, only adder

In [8]:
questions = []
expected = []
seen = set()

print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                            for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))

    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))



Generating data...
Total addition questions: 50000


In [9]:
print(questions[:5], expected[:5])


['5+3    ', '10+360 ', '711+706', '3+385  ', '2+871  '] ['8   ', '370 ', '1417', '388 ', '873 ']


# Data Processing

In [10]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)
print('Done...')

Vectorization...


# Shuffle & Split train, test, validation data

In [12]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:30000]
train_y = y[:30000]
test_x = x[30000:]
test_y = y[30000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)



Training Data:
(27000, 7, 12)
(27000, 4, 12)
Validation Data:
(3000, 7, 12)
(3000, 4, 12)
Testing Data:
(20000, 7, 12)
(20000, 4, 12)


# Build Model

In [21]:
print('Build model...')

############################################
##### Build your own model here ############
model = Sequential()

# "Encode" the input sequence using an RNN, input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))

# the decoder RNN's input
model.add(layers.RepeatVector(DIGITS + 1))

# The decoder RNN  with a single layer.
for _ in range(LAYERS):
    # all the outputs in the form of (num_samples, timesteps, output_dim). 
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))


model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))

############################################
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()


Build model...
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [22]:
for iteration in range(50):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=3,
              validation_data=(x_val, y_val))
    # visualizing validation set prediction errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)



--------------------------------------------------
Iteration 0
Train on 27000 samples, validate on 3000 samples
Epoch 1/3
27000/27000 [==============================] - 14s 502us/step - loss: 1.9426 - acc: 0.3091 - val_loss: 1.8398 - val_acc: 0.3408
Epoch 2/3
27000/27000 [==============================] - 7s 265us/step - loss: 1.8078 - acc: 0.3394 - val_loss: 1.7888 - val_acc: 0.3408
Epoch 3/3
27000/27000 [==============================] - 8s 286us/step - loss: 1.7621 - acc: 0.3485 - val_loss: 1.7360 - val_acc: 0.3645
Q   8+922 T 237  ☒ 111 
Q   2+264 T 464  ☒ 31  
Q  118+25 T 863  ☒ 301 
Q   7+555 T 562  ☒ 666 
Q  42+498 T 918  ☒ 900 
Q  468+22 T 886  ☒ 393 
Q   9+027 T 729  ☒ 101 
Q  26+318 T 875  ☒ 801 
Q  695+87 T 674  ☒ 906 
Q   4+115 T 515  ☒ 11  

--------------------------------------------------
Iteration 1
Train on 27000 samples, validate on 3000 samples
Epoch 1/3
27000/27000 [==============================] - 7s 263us/step - loss: 1.6729 - acc: 0.3802 - val_loss: 1.6155 - v

Q  668+87 T 944  ☑ 944 
Q 337+786 T 1420 ☑ 1420
Q  029+47 T 994  ☒ 993 
Q 062+595 T 855  ☑ 855 
Q   61+71 T 33   ☑ 33  
Q   22+13 T 53   ☑ 53  
Q 544+696 T 1141 ☒ 1131
Q 621+214 T 538  ☒ 528 
Q  18+357 T 834  ☑ 834 
Q   0+648 T 846  ☑ 846 

--------------------------------------------------
Iteration 10
Train on 27000 samples, validate on 3000 samples
Epoch 1/3
27000/27000 [==============================] - 8s 286us/step - loss: 0.1450 - acc: 0.9649 - val_loss: 0.1850 - val_acc: 0.9419
Epoch 2/3
27000/27000 [==============================] - 8s 295us/step - loss: 0.1316 - acc: 0.9685 - val_loss: 0.1729 - val_acc: 0.9447
Epoch 3/3
27000/27000 [==============================] - 8s 296us/step - loss: 0.1130 - acc: 0.9760 - val_loss: 0.1284 - val_acc: 0.9643
Q   255+2 T 554  ☑ 554 
Q   239+9 T 941  ☑ 941 
Q 947+666 T 1415 ☑ 1415
Q  28+185 T 663  ☑ 663 
Q 044+214 T 852  ☑ 852 
Q  66+996 T 765  ☑ 765 
Q   1+754 T 458  ☑ 458 
Q  32+032 T 253  ☑ 253 
Q  91+018 T 829  ☑ 829 
Q  52+404 T 429  ☑ 

27000/27000 [==============================] - 8s 303us/step - loss: 0.0501 - acc: 0.9835 - val_loss: 0.0293 - val_acc: 0.9901
Epoch 2/3
27000/27000 [==============================] - 8s 283us/step - loss: 0.0058 - acc: 0.9995 - val_loss: 0.0224 - val_acc: 0.9920
Epoch 3/3
27000/27000 [==============================] - 7s 262us/step - loss: 0.0034 - acc: 0.9999 - val_loss: 0.0206 - val_acc: 0.9929
Q  458+54 T 899  ☑ 899 
Q   265+5 T 567  ☑ 567 
Q  95+906 T 668  ☑ 668 
Q   408+5 T 809  ☑ 809 
Q  157+32 T 774  ☑ 774 
Q 673+855 T 934  ☑ 934 
Q   454+6 T 460  ☑ 460 
Q  733+96 T 406  ☑ 406 
Q  15+967 T 820  ☑ 820 
Q 044+214 T 852  ☑ 852 

--------------------------------------------------
Iteration 30
Train on 27000 samples, validate on 3000 samples
Epoch 1/3
27000/27000 [==============================] - 7s 261us/step - loss: 0.0028 - acc: 1.0000 - val_loss: 0.0188 - val_acc: 0.9934
Epoch 2/3
27000/27000 [==============================] - 5s 198us/step - loss: 0.0026 - acc: 1.0000 - val_lo

Q  118+25 T 863  ☑ 863 
Q  31+567 T 778  ☑ 778 
Q 383+621 T 509  ☑ 509 
Q  638+36 T 899  ☒ 999 
Q  884+64 T 534  ☑ 534 

--------------------------------------------------
Iteration 39
Train on 27000 samples, validate on 3000 samples
Epoch 1/3
27000/27000 [==============================] - 4s 140us/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.0151 - val_acc: 0.9951
Epoch 2/3
27000/27000 [==============================] - 4s 142us/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.0141 - val_acc: 0.9959
Epoch 3/3
27000/27000 [==============================] - 4s 141us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0147 - val_acc: 0.9953
Q   9+281 T 191  ☑ 191 
Q   0+202 T 202  ☑ 202 
Q  041+08 T 220  ☑ 220 
Q 754+766 T 1124 ☑ 1124
Q   605+4 T 510  ☑ 510 
Q  28+375 T 655  ☑ 655 
Q  014+49 T 504  ☑ 504 
Q 722+021 T 347  ☑ 347 
Q 469+767 T 1731 ☑ 1731
Q 685+004 T 986  ☑ 986 

--------------------------------------------------
Iteration 40
Train on 27000 samples, validate on 3000 samples
E

# Testing

In [23]:
*print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 

preds = model.predict_classes(test_x, verbose=0)

i = 0
count = 0
while i < len(preds) :
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)

    print('Q', q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        count += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
    i += 1
    
testing_acc = (count/len(test_y))*100
print('-'*60)
print('-'*30, colors.ok + str(testing_acc) + "%" + colors.close, '-'*30)
print('-'*60)
#####################################################


MSG : Prediction
Q 455+93  T 548  ☑ 548 
Q 589+3   T 592  ☑ 592 
Q 164+98  T 262  ☑ 262 
Q 716+10  T 726  ☑ 726 
Q 630+3   T 633  ☑ 633 
Q 83+579  T 662  ☑ 662 
Q 12+463  T 475  ☑ 475 
Q 519+560 T 1079 ☑ 1079
Q 858+16  T 874  ☑ 874 
Q 795+40  T 835  ☑ 835 
Q 922+6   T 928  ☑ 928 
Q 624+49  T 673  ☑ 673 
Q 55+261  T 316  ☑ 316 
Q 555+451 T 1006 ☑ 1006
Q 47+5    T 52   ☑ 52  
Q 963+9   T 972  ☑ 972 
Q 0+427   T 427  ☑ 427 
Q 603+78  T 681  ☑ 681 
Q 119+284 T 403  ☑ 403 
Q 27+259  T 286  ☑ 286 
Q 38+43   T 81   ☑ 81  
Q 22+996  T 1018 ☑ 1018
Q 53+285  T 338  ☑ 338 
Q 686+99  T 785  ☑ 785 
Q 352+19  T 371  ☑ 371 
Q 867+521 T 1388 ☑ 1388
Q 40+388  T 428  ☑ 428 
Q 438+101 T 539  ☑ 539 
Q 682+4   T 686  ☑ 686 
Q 338+43  T 381  ☑ 381 
Q 405+4   T 409  ☑ 409 
Q 26+5    T 31   ☑ 31  
Q 45+700  T 745  ☑ 745 
Q 97+854  T 951  ☑ 951 
Q 555+135 T 690  ☑ 690 
Q 335+67  T 402  ☑ 402 
Q 494+9   T 503  ☑ 503 
Q 11+4    T 15   ☑ 15  
Q 22+909  T 931  ☑ 931 
Q 866+41  T 907  ☑ 907 
Q 533+5   T 538  ☑ 538 

Q 946+114 T 1060 ☑ 1060
Q 125+54  T 179  ☑ 179 
Q 727+856 T 1583 ☑ 1583
Q 35+96   T 131  ☑ 131 
Q 50+889  T 939  ☑ 939 
Q 434+951 T 1385 ☑ 1385
Q 18+649  T 667  ☑ 667 
Q 560+5   T 565  ☑ 565 
Q 87+894  T 981  ☑ 981 
Q 677+84  T 761  ☑ 761 
Q 720+888 T 1608 ☒ 1508
Q 937+45  T 982  ☑ 982 
Q 628+5   T 633  ☑ 633 
Q 836+323 T 1159 ☑ 1159
Q 848+45  T 893  ☑ 893 
Q 53+231  T 284  ☑ 284 
Q 11+881  T 892  ☑ 892 
Q 8+530   T 538  ☑ 538 
Q 610+65  T 675  ☑ 675 
Q 73+852  T 925  ☑ 925 
Q 205+111 T 316  ☑ 316 
Q 154+31  T 185  ☑ 185 
Q 815+956 T 1771 ☑ 1771
Q 665+151 T 816  ☑ 816 
Q 5+454   T 459  ☑ 459 
Q 742+422 T 1164 ☑ 1164
Q 95+616  T 711  ☑ 711 
Q 740+65  T 805  ☑ 805 
Q 49+99   T 148  ☑ 148 
Q 51+707  T 758  ☑ 758 
Q 211+2   T 213  ☑ 213 
Q 391+397 T 788  ☑ 788 
Q 2+751   T 753  ☑ 753 
Q 5+314   T 319  ☑ 319 
Q 343+563 T 906  ☑ 906 
Q 364+774 T 1138 ☑ 1138
Q 1+294   T 295  ☑ 295 
Q 412+333 T 745  ☑ 745 
Q 189+672 T 861  ☑ 861 
Q 60+18   T 78   ☑ 78  
Q 61+203  T 264  ☑ 264 
Q 94+914  T 1008

Q 968+66  T 1034 ☑ 1034
Q 871+16  T 887  ☑ 887 
Q 863+627 T 1490 ☑ 1490
Q 856+20  T 876  ☑ 876 
Q 109+403 T 512  ☑ 512 
Q 518+152 T 670  ☑ 670 
Q 995+294 T 1289 ☑ 1289
Q 2+387   T 389  ☑ 389 
Q 94+413  T 507  ☑ 507 
Q 37+22   T 59   ☑ 59  
Q 587+349 T 936  ☑ 936 
Q 50+195  T 245  ☑ 245 
Q 845+87  T 932  ☑ 932 
Q 868+336 T 1204 ☑ 1204
Q 363+421 T 784  ☑ 784 
Q 391+437 T 828  ☑ 828 
Q 9+785   T 794  ☑ 794 
Q 315+908 T 1223 ☑ 1223
Q 17+580  T 597  ☑ 597 
Q 108+5   T 113  ☑ 113 
Q 752+601 T 1353 ☑ 1353
Q 725+924 T 1649 ☑ 1649
Q 15+731  T 746  ☑ 746 
Q 42+882  T 924  ☑ 924 
Q 3+107   T 110  ☑ 110 
Q 1+372   T 373  ☑ 373 
Q 87+122  T 209  ☑ 209 
Q 159+56  T 215  ☑ 215 
Q 29+708  T 737  ☑ 737 
Q 49+23   T 72   ☑ 72  
Q 55+772  T 827  ☑ 827 
Q 29+69   T 98   ☑ 98  
Q 655+95  T 750  ☑ 750 
Q 455+233 T 688  ☑ 688 
Q 8+435   T 443  ☑ 443 
Q 25+519  T 544  ☑ 544 
Q 45+480  T 525  ☑ 525 
Q 41+454  T 495  ☑ 495 
Q 37+361  T 398  ☑ 398 
Q 38+565  T 603  ☑ 603 
Q 560+75  T 635  ☑ 635 
Q 0+448   T 448 

Q 50+11   T 61   ☑ 61  
Q 88+863  T 951  ☑ 951 
Q 30+900  T 930  ☑ 930 
Q 81+883  T 964  ☑ 964 
Q 7+451   T 458  ☑ 458 
Q 339+82  T 421  ☑ 421 
Q 815+162 T 977  ☑ 977 
Q 860+939 T 1799 ☑ 1799
Q 3+551   T 554  ☑ 554 
Q 1+878   T 879  ☑ 879 
Q 865+354 T 1219 ☑ 1219
Q 966+42  T 1008 ☑ 1008
Q 29+64   T 93   ☑ 93  
Q 13+91   T 104  ☑ 104 
Q 913+744 T 1657 ☑ 1657
Q 12+13   T 25   ☑ 25  
Q 945+82  T 1027 ☑ 1027
Q 449+72  T 521  ☑ 521 
Q 225+605 T 830  ☑ 830 
Q 495+923 T 1418 ☑ 1418
Q 417+43  T 460  ☑ 460 
Q 59+968  T 1027 ☑ 1027
Q 875+2   T 877  ☑ 877 
Q 538+1   T 539  ☑ 539 
Q 360+769 T 1129 ☑ 1129
Q 673+7   T 680  ☑ 680 
Q 79+27   T 106  ☑ 106 
Q 819+39  T 858  ☑ 858 
Q 591+266 T 857  ☑ 857 
Q 58+663  T 721  ☑ 721 
Q 945+59  T 1004 ☑ 1004
Q 241+17  T 258  ☑ 258 
Q 66+162  T 228  ☑ 228 
Q 3+237   T 240  ☑ 240 
Q 983+543 T 1526 ☑ 1526
Q 7+576   T 583  ☑ 583 
Q 510+12  T 522  ☑ 522 
Q 315+498 T 813  ☑ 813 
Q 8+250   T 258  ☑ 258 
Q 73+376  T 449  ☑ 449 
Q 333+40  T 373  ☑ 373 
Q 63+974  T 1037

Q 629+557 T 1186 ☑ 1186
Q 98+41   T 139  ☑ 139 
Q 60+954  T 1014 ☑ 1014
Q 406+2   T 408  ☑ 408 
Q 398+21  T 419  ☑ 419 
Q 590+0   T 590  ☑ 590 
Q 778+75  T 853  ☑ 853 
Q 670+57  T 727  ☑ 727 
Q 40+897  T 937  ☑ 937 
Q 68+865  T 933  ☑ 933 
Q 60+44   T 104  ☑ 104 
Q 69+299  T 368  ☑ 368 
Q 87+952  T 1039 ☑ 1039
Q 885+775 T 1660 ☑ 1660
Q 72+938  T 1010 ☑ 1010
Q 38+56   T 94   ☑ 94  
Q 61+25   T 86   ☑ 86  
Q 81+717  T 798  ☑ 798 
Q 625+9   T 634  ☑ 634 
Q 4+436   T 440  ☑ 440 
Q 43+725  T 768  ☑ 768 
Q 896+41  T 937  ☑ 937 
Q 11+93   T 104  ☑ 104 
Q 925+438 T 1363 ☑ 1363
Q 9+522   T 531  ☑ 531 
Q 323+19  T 342  ☑ 342 
Q 340+586 T 926  ☑ 926 
Q 439+948 T 1387 ☑ 1387
Q 275+468 T 743  ☑ 743 
Q 20+90   T 110  ☒ 100 
Q 292+127 T 419  ☑ 419 
Q 517+40  T 557  ☑ 557 
Q 569+811 T 1380 ☑ 1380
Q 25+600  T 625  ☑ 625 
Q 380+0   T 380  ☑ 380 
Q 71+75   T 146  ☑ 146 
Q 334+1   T 335  ☑ 335 
Q 26+238  T 264  ☑ 264 
Q 720+151 T 871  ☑ 871 
Q 25+874  T 899  ☑ 899 
Q 80+453  T 533  ☑ 533 
Q 47+938  T 985 

...

Q 435+85  T 520  ☑ 520 
Q 356+4   T 360  ☑ 360 
Q 11+414  T 425  ☑ 425 
Q 128+705 T 833  ☑ 833 
Q 195+1   T 196  ☑ 196 
Q 94+113  T 207  ☑ 207 
Q 19+802  T 821  ☑ 821 
Q 436+768 T 1204 ☑ 1204
Q 941+609 T 1550 ☑ 1550
Q 655+274 T 929  ☑ 929 
Q 823+245 T 1068 ☑ 1068
Q 84+958  T 1042 ☑ 1042
Q 752+24  T 776  ☑ 776 
Q 531+450 T 981  ☑ 981 
Q 76+223  T 299  ☑ 299 
Q 865+84  T 949  ☑ 949 
Q 10+729  T 739  ☑ 739 
Q 720+595 T 1315 ☑ 1315
Q 64+442  T 506  ☑ 506 
Q 10+776  T 786  ☑ 786 
Q 32+14   T 46   ☑ 46  
Q 297+8   T 305  ☑ 305 
Q 4+783   T 787  ☑ 787 
Q 33+699  T 732  ☑ 732 
Q 596+551 T 1147 ☑ 1147
Q 334+39  T 373  ☑ 373 
Q 415+403 T 818  ☑ 818 
Q 632+3   T 635  ☑ 635 
Q 647+99  T 746  ☑ 746 
Q 37+359  T 396  ☑ 396 
Q 209+74  T 283  ☑ 283 
Q 469+1   T 470  ☑ 470 
Q 81+668  T 749  ☑ 749 
Q 759+234 T 993  ☑ 993 
Q 634+841 T 1475 ☑ 1475
Q 207+617 T 824  ☑ 824 
Q 2+164   T 166  ☑ 166 
Q 286+56  T 342  ☑ 342 
Q 21+948  T 969  ☑ 969 
Q 40+565  T 605  ☑ 605 
Q 360+752 T 1112 ☑ 1112
Q 361+76  T 437 

Q 29+902  T 931  ☑ 931 
Q 60+293  T 353  ☑ 353 
Q 163+409 T 572  ☑ 572 
Q 7+646   T 653  ☑ 653 
Q 105+16  T 121  ☑ 121 
Q 999+2   T 1001 ☑ 1001
Q 63+788  T 851  ☑ 851 
Q 8+30    T 38   ☑ 38  
Q 79+155  T 234  ☑ 234 
Q 16+269  T 285  ☑ 285 
Q 80+602  T 682  ☑ 682 
Q 979+871 T 1850 ☑ 1850
Q 11+273  T 284  ☑ 284 
Q 460+37  T 497  ☑ 497 
Q 292+35  T 327  ☑ 327 
Q 303+534 T 837  ☑ 837 
Q 472+8   T 480  ☑ 480 
Q 420+961 T 1381 ☑ 1381
Q 90+12   T 102  ☑ 102 
Q 199+71  T 270  ☑ 270 
Q 686+989 T 1675 ☑ 1675
Q 703+60  T 763  ☑ 763 
Q 20+499  T 519  ☑ 519 
Q 71+342  T 413  ☑ 413 
Q 45+196  T 241  ☑ 241 
Q 69+377  T 446  ☑ 446 
Q 758+210 T 968  ☑ 968 
Q 62+1    T 63   ☑ 63  
Q 16+90   T 106  ☑ 106 
Q 690+37  T 727  ☑ 727 
Q 826+2   T 828  ☑ 828 
Q 99+621  T 720  ☑ 720 
Q 470+478 T 948  ☑ 948 
Q 265+617 T 882  ☑ 882 
Q 641+927 T 1568 ☑ 1568
Q 901+11  T 912  ☑ 912 
Q 958+840 T 1798 ☑ 1798
Q 29+235  T 264  ☑ 264 
Q 731+70  T 801  ☑ 801 
Q 775+881 T 1656 ☑ 1656
Q 74+815  T 889  ☑ 889 
Q 261+257 T 518 

Q 910+32  T 942  ☑ 942 
Q 189+21  T 210  ☑ 210 
Q 863+678 T 1541 ☑ 1541
Q 868+352 T 1220 ☑ 1220
Q 817+270 T 1087 ☑ 1087
Q 261+18  T 279  ☑ 279 
Q 6+94    T 100  ☑ 100 
Q 79+626  T 705  ☑ 705 
Q 80+51   T 131  ☑ 131 
Q 47+832  T 879  ☑ 879 
Q 934+23  T 957  ☑ 957 
Q 765+68  T 833  ☑ 833 
Q 175+362 T 537  ☑ 537 
Q 94+25   T 119  ☑ 119 
Q 59+972  T 1031 ☑ 1031
Q 42+55   T 97   ☑ 97  
Q 469+99  T 568  ☑ 568 
Q 802+8   T 810  ☑ 810 
Q 62+36   T 98   ☑ 98  
Q 55+926  T 981  ☑ 981 
Q 322+7   T 329  ☑ 329 
Q 84+867  T 951  ☑ 951 
Q 282+77  T 359  ☑ 359 
Q 93+92   T 185  ☑ 185 
Q 377+41  T 418  ☑ 418 
Q 384+988 T 1372 ☑ 1372
Q 207+87  T 294  ☑ 294 
Q 962+1   T 963  ☑ 963 
Q 353+776 T 1129 ☑ 1129
Q 585+288 T 873  ☑ 873 
Q 46+468  T 514  ☑ 514 
Q 2+482   T 484  ☑ 484 
Q 72+224  T 296  ☑ 296 
Q 207+27  T 234  ☑ 234 
Q 514+924 T 1438 ☑ 1438
Q 472+247 T 719  ☑ 719 
Q 240+988 T 1228 ☑ 1228
Q 832+9   T 841  ☑ 841 
Q 504+70  T 574  ☑ 574 
Q 56+39   T 95   ☑ 95  
Q 411+334 T 745  ☑ 745 
Q 60+259  T 319 

Q 84+78   T 162  ☑ 162 
Q 834+67  T 901  ☑ 901 
Q 1+454   T 455  ☑ 455 
Q 288+492 T 780  ☑ 780 
Q 63+992  T 1055 ☑ 1055
Q 268+6   T 274  ☑ 274 
Q 185+33  T 218  ☑ 218 
Q 15+21   T 36   ☑ 36  
Q 84+48   T 132  ☑ 132 
Q 28+390  T 418  ☑ 418 
Q 58+44   T 102  ☑ 102 
Q 69+778  T 847  ☑ 847 
Q 513+483 T 996  ☑ 996 
Q 9+497   T 506  ☑ 506 
Q 506+94  T 600  ☑ 600 
Q 444+657 T 1101 ☑ 1101
Q 898+896 T 1794 ☑ 1794
Q 51+865  T 916  ☑ 916 
Q 587+13  T 600  ☑ 600 
Q 992+6   T 998  ☑ 998 
Q 8+610   T 618  ☑ 618 
Q 78+993  T 1071 ☑ 1071
Q 461+29  T 490  ☑ 490 
Q 6+915   T 921  ☑ 921 
Q 133+39  T 172  ☑ 172 
Q 94+509  T 603  ☑ 603 
Q 503+78  T 581  ☑ 581 
Q 922+4   T 926  ☑ 926 
Q 834+22  T 856  ☑ 856 
Q 9+728   T 737  ☑ 737 
Q 16+782  T 798  ☑ 798 
Q 670+505 T 1175 ☑ 1175
Q 89+99   T 188  ☑ 188 
Q 724+75  T 799  ☑ 799 
Q 914+868 T 1782 ☑ 1782
Q 93+990  T 1083 ☑ 1083
Q 734+55  T 789  ☑ 789 
Q 405+978 T 1383 ☑ 1383
Q 8+118   T 126  ☑ 126 
Q 792+315 T 1107 ☒ 1007
Q 78+374  T 452  ☑ 452 
Q 695+1   T 696 

Q 472+4   T 476  ☑ 476 
Q 655+121 T 776  ☑ 776 
Q 7+966   T 973  ☑ 973 
Q 139+7   T 146  ☑ 146 
Q 238+991 T 1229 ☑ 1229
Q 57+43   T 100  ☑ 100 
Q 635+62  T 697  ☑ 697 
Q 612+72  T 684  ☑ 684 
Q 22+146  T 168  ☑ 168 
Q 93+682  T 775  ☑ 775 
Q 552+660 T 1212 ☑ 1212
Q 696+769 T 1465 ☑ 1465
Q 975+65  T 1040 ☑ 1040
Q 897+8   T 905  ☑ 905 
Q 40+463  T 503  ☑ 503 
Q 76+307  T 383  ☑ 383 
Q 5+304   T 309  ☑ 309 
Q 55+43   T 98   ☑ 98  
Q 896+454 T 1350 ☒ 1340
Q 31+844  T 875  ☑ 875 
Q 98+718  T 816  ☑ 816 
Q 39+967  T 1006 ☑ 1006
Q 399+491 T 890  ☑ 890 
Q 239+2   T 241  ☑ 241 
Q 28+950  T 978  ☑ 978 
Q 874+237 T 1111 ☑ 1111
Q 4+699   T 703  ☑ 703 
Q 910+43  T 953  ☑ 953 
Q 38+13   T 51   ☑ 51  
Q 418+7   T 425  ☑ 425 
Q 21+456  T 477  ☑ 477 
Q 6+558   T 564  ☑ 564 
Q 519+161 T 680  ☑ 680 
Q 1+295   T 296  ☑ 296 
Q 724+997 T 1721 ☑ 1721
Q 21+247  T 268  ☑ 268 
Q 642+122 T 764  ☑ 764 
Q 451+48  T 499  ☑ 499 
Q 150+2   T 152  ☑ 152 
Q 92+589  T 681  ☑ 681 
Q 240+959 T 1199 ☑ 1199
Q 939+64  T 1003